## Práctica Guiada II - Kmeans Clustering y reducción de dimensiones sobre texto

En esta práctica guiada vamos a aplicar el algoritmo kmeans a textos de distintos tópicos para verificar si el agrupamiento coincide con las categorías que se les dieron a los mismos.

* Primero vamos a pasar de los features de texto a un bag of words.
* Luego vamos reducir la dimensionadlidad del bag of words con una técnica de reducción especial para trabajar con espacios súper-dispersos llamada <a href='http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html'>Truncated SVD </a>
* Por último vamos a aplicar Kmeans con tantos clusters como clases había inicialmente para luego visualizar los datos y calcular métricas de validación.

In [4]:
# Importamos librerías
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import numpy as np


### 1 - Construcción del dataset

Importamos el dataset newsgroup20 con emails clasificados en distintas categorías. Seleccionamos específicamente 3 de las categorías existentes. 



In [5]:
# #############################################################################
# Cargamos las categorías
categories = [
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

print("Cargando categorías:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d documentos" % len(dataset.data))
print("%d categorías" % len(dataset.target_names))
print()

labels = dataset.target
true_k = np.unique(labels).shape[0]

Cargando categorías:
['talk.religion.misc', 'comp.graphics', 'sci.space']
2588 documentos
3 categorías



In [6]:
true_k

3

### 2 - Vectorización y Bag of Words

A continuación construimos el Bag of Words, limitando la ocurrencia de las palabras con max-df y min-df, deshaciéndonos de las stop words y con un máximo de 10000 features.

In [7]:
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english',
                             use_idf=True)

X = vectorizer.fit_transform(dataset.data)

print("n_samples: %d, n_features: %d" % X.shape)
print()


n_samples: 2588, n_features: 10000



### 3 - Reducción de dimensiones

Para la reducción de dimensiones usamos TruncatedSVD(). Los resultados de este algoritmo no están normalizados, y para el buen funcionamiento de kmeans es importante que todos los features se encuentren en la misma escala, por ser un algoritmo basado en la distancia.
El método Normalizer() de sklearn reescala todos los features para tener norma l2 igual a 1.

In [8]:
svd = TruncatedSVD(100,random_state = 1)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(X)


### 4 - Clustering

Con este preprocesamiento, estamos listos para aplicar kmeans. Vamos a utilizar 3 centroides para ver si podemos detectar la estructura de las categorías originales.

In [9]:
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
            verbose=1, random_state = 1)

print("Clustering sparse data with %s" % km)
km.fit(X)
print()



Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=1)
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 2310.24120137
start iteration
done sorting
end inner loop
Iteration 1, inertia 2256.39380889
start iteration
done sorting
end inner loop
Iteration 2, inertia 2242.86662115
start iteration
done sorting
end inner loop
Iteration 3, inertia 2241.0565545
start iteration
done sorting
end inner loop
Iteration 4, inertia 2240.64812454
start iteration
done sorting
end inner loop
Iteration 5, inertia 2240.06951145
start iteration
done sorting
end inner loop
Iteration 6, inertia 2238.31032003
start iteration
done sorting
end inner loop
Iteration 7, inertia 2236.70447913
start iteration
done sorting
end inner loop
Iteration 8, inertia 2235.42778694
start iteration
done sorting
end inner loop
Iteration 9, i

### 5 - Validación de resultados

Lo primero que vamos a hacer para explorar los resultados es volver atrás en la reducción de dimensiones para verificar cuáles son las palabras que con más fuerza componen cada uno de los clusters.

In [10]:
print("Ranking de términos por cluster:")

# Los centroides están mapeados en el espacio de dimeniosnes reducidas.
# Hacemos la transformación inversa para ver el peso de los features originales.
original_space_centroids = svd.inverse_transform(km.cluster_centers_)

# La función argsort nos da los índices ordenados
order_centroids = original_space_centroids.argsort()[:, ::-1]


# Imprimimos los términos más "pesados" (frecuentes) para cada categoría.
# Con los índices ordenados, llamamos a la descripción de las palabras que construyó el Vectorizer()
terms = vectorizer.get_feature_names()

for i in range(true_k):
    
    print("Cluster %d:" % i, end='')
    
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    
    print()


Ranking de términos por cluster:
Cluster 0: space nasa access henry com digex pat gov toronto shuttle
Cluster 1: god com jesus people christian sandvik don bible article say
Cluster 2: graphics university thanks image com files help 3d posting computer


### Silhouette score

In [11]:
from sklearn.metrics import silhouette_score
silhouette_score(X, km.labels_)

0.040026946088605443

Recordemos que un Silhouette Score cercano a 0 indica que hay clusters que se superponen.

### Medidas de validación externa

En base a los términos encontrados en los clusters podríamos concluir que 0 es 'sci.space', el 1 'talk.religion' y el 2 es 'comp.graphics'.
Con esto podemos calcular el cluster predicho para cada noticia.

In [12]:
dataset.target_names

['comp.graphics', 'sci.space', 'talk.religion.misc']

In [13]:
predY = np.choose(dataset.target, [2, 0, 1]).astype(np.int64)

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(predY,km.labels_)

array([[919,   7,  61],
       [ 15, 598,  15],
       [ 18,  21, 934]])

In [15]:
accuracy_score(predY,km.labels_)

0.94706336939721791